In [1]:
import numpy as np
import pandas as pd
from ortools.sat.python import cp_model

df = pd.read_csv("data/players.csv")

#### Set the constraints
1. We need 2 goalkeepers, 5 defenders, 5 midfielders and 3 forwards
2. We cannot spend over $100m
3. We can pick no more than 3 players from the same team

Constraint #1

In [2]:
model = cp_model.CpModel()

# Define the number of players we want to select in each position
POSITION_MAP = {
    "Goalkeeper": {"code": "GKP", "count": 2},
    "Defender": {"code": "DEF", "count": 5},
    "Midfielder": {"code": "MID", "count": 5},
    "Forward": {"code": "FWD", "count": 3}
}

decision_variables = {}

In [3]:
# add constraints to each position
for position, details in POSITION_MAP.items():
    
    # find all eligible players, i.e. players who play the desired position
    players_in_position = list(df[df["position"] == details["code"]].id.values)

    # fetch the number of players we're allowed to pick in this position
    player_count = details["count"]

    # create decision variables for each of the players in that position
    player_variables = {i: model.NewBoolVar(f"player{i}") for i in players_in_position}
    decision_variables.update(player_variables) # add decision variable to global dict

    # add the constraint: per position, only allowed the specified count of players
    model.Add(sum(player_variables.values()) == player_count)

Constraint #2

In [4]:
BUDGET = 1000
player_costs = {player: df[df["id"] == player]["now_cost"].values[0] for player in df["id"]}
model.Add(sum(var * player_costs[i] for i, var in decision_variables.items()) <= BUDGET)

Constraint #3

In [5]:
MAX_PLAYERS_PER_TEAM = 3
teams = df["team"].unique()
for team in teams:
    eligible_players = df[df["team"] == team].id.values
    model.Add(sum(decision_variables[i] for i in eligible_players) <= MAX_PLAYERS_PER_TEAM)

#### Add objective function: maximize "points_per_game"

In [6]:
# for each player, collect their average "points_per_game"
player_points_per_game = {player: df[df["id"] == player]["points_per_game"].values[0] for player in decision_variables.keys()}

# add another decision variable: the total "points_per_game" of the selected players
# this will be the sum of (points_per_game)*(1 or 0) for each player depending on whether the player is selected
total_points = sum(var * player_points_per_game[i] for i, var in decision_variables.items())

# intruct model to maximize this value
model.Maximize(total_points)

#### Solve

In [7]:
solver = cp_model.CpSolver()
status = solver.Solve(model)

In [8]:
def show_solution(status):
    """
    Print out the solution if any exists
    """
    if status == cp_model.OPTIMAL:
        print("Optimal solution found. Players selected:\n")
        total_cost = 0
        total_points_per_game = 0

        players = {
            "GKP": [],
            "DEF": [],
            "MID": [],
            "FWD": []
        }
        for i, var in decision_variables.items():
            if solver.Value(var) == 1:
                player_position = df[df["id"] == i].position.values[0]
                players[player_position].append(i)
        
        for position, ids in players.items():
            print(f"\nPlayers in {position}:")
            for i in ids:
                player_name = df[df["id"] == i]["name"].values[0]
                player_cost = df[df["id"] == i]["now_cost"].values[0]
                player_team = df[df["id"] == i]["team"].values[0]
                player_points_per_game = df[df["id"] == i]["points_per_game"].values[0]
                print(f"{player_name}: {player_team}, ${player_cost}, {player_points_per_game}")
                total_cost += player_cost
                total_points_per_game += player_points_per_game
            
        print("\nTotal cost: ", total_cost)
        print("\nTotal points per game: ", total_points_per_game)
    
    else:
        print("No solution found.")

In [9]:
show_solution(status)

Optimal solution found. Players selected:


Players in GKP:
Lukasz Fabianski: West Ham, $42, 4.0
Daniel Bentley: Wolves, $39, 5.3

Players in DEF:
Seamus Coleman: Everton, $44, 6.0
Vitalii Mykolenko: Everton, $46, 4.8
Trent Alexander-Arnold: Liverpool, $83, 5.6
Konstantinos Tsimikas: Liverpool, $48, 4.5
Sven Botman: Newcastle, $45, 4.6

Players in MID:
Bryan Mbeumo: Brentford, $68, 5.8
Solly March: Brighton, $61, 5.6
André Tavares Gomes: Everton, $44, 7.0
Son Heung-min: Spurs, $98, 6.8
Jarrod Bowen: West Ham, $78, 6.0

Players in FWD:
Ollie Watkins: Aston Villa, $88, 6.1
Christopher Nkunku: Chelsea, $73, 7.0
Erling Haaland: Man City, $139, 7.5

Total cost:  996

Total points per game:  86.6
